# IMPORTACIÓN DE LIBRERÍAS.

Se importan la librería de pandas para manipular marcos de datos (dataframes) y numpy para cálculos matemáticos en caso de ser estos necesarios.

In [1]:
import pandas as pd
import numpy as np
import datetime

# IMPORTACIÓN DE CATÁLOGOS.

Para conocer el medio, las televisoras, deflactación en inversión, TRPs 20', la categoría y subcategoría de nuestras marcas y productos propios y de la competencia.

In [2]:
ruta_catalogos_AR = 'C:\\Users\\brandon.olvera\\OneDrive - insidemedia.net\\Documents\\Bayer\\2023\\LATAM\\Catalogos\\AG\\Catalogos ARG Layout v2.xlsx'
Catalogos_AR = pd.read_excel(ruta_catalogos_AR, sheet_name=None)
Cat_Franj_AR=Catalogos_AR['Cat_Franj_AR']
CatBrandPr_AR = Catalogos_AR['CatBrandPr_AR']
CatDescu_AR = Catalogos_AR['CatDescu_AR']
CatMediaTv_AR=Catalogos_AR['CatMediaTv_AR']
CatMediaNoTV_AR=Catalogos_AR['CatMediaNoTV_AR']
Cat_Targ_by_BC_AR=Catalogos_AR['Cat_Targ_by_BC_AR']
Info_Targets_AR=Catalogos_AR['Info_Targets_AR']
Cat_Anunc_AR=Catalogos_AR['Cat_Anunc_AR']

# IMPORTACIÓN DE INFORMACIÓN.

En esta sección se cargan los reportes de MSS Adspend (Radio y Print) y MSS TV (OTV y PTV) y se les asignan nombres para poder hacer las manipulaciones correspondientes. Esta entrada tendrá que ser modificada cada vez que se quiera utilizar distintos reportes de MSS.

In [3]:
ruta_base_AR = 'C:\\Users\\brandon.olvera\\OneDrive - insidemedia.net\\Documents\\Bayer\\2023\\LATAM\\Bases x Mercado\\2023\\Base_AR_2023_01.xlsx'
Dict_Base_raw_AR=pd.read_excel(ruta_base_AR,sheet_name=None)
Base_raw_AR=Dict_Base_raw_AR["Data"]

# LIMPIEZA

En esta sección se agregarán las columnas nuevas que nos permitirán categorizar nuestras marcas y productos. Deflactar cifras de inversión y TRPs homologados.

In [4]:
BAYER_AR=Base_raw_AR.merge(CatBrandPr_AR,how="left",left_on=["Marca","Producto"],right_on=["MARCA","PRODUCTO"])
BAYER_AR=BAYER_AR.merge(CatDescu_AR,how="left",left_on=["Anio","Clase De Vehiculo"],right_on=["Año","Medio Descuento"])
BAYER_AR=BAYER_AR.merge(CatMediaNoTV_AR,how="left",left_on="Clase De Vehiculo",right_on="Medio")
BAYER_AR=BAYER_AR.merge(CatMediaTv_AR,how="left",left_on="Vehiculo",right_on="Canal")
BAYER_AR["Tipo de Medio"] = BAYER_AR["Tipo de Medio_x"].fillna(BAYER_AR["Tipo de Medio_y"])
BAYER_AR["Medio Descuento"] = BAYER_AR["Medio Descuento_x"].fillna(BAYER_AR["Medio Descuento_y"])
BAYER_AR["Inversion"]=BAYER_AR["Inversion"].astype(float)
BAYER_AR["Miles"]=BAYER_AR["Inversion"]/1000
BAYER_AR["Miles con Descuento"]=BAYER_AR["Miles"]*(1-BAYER_AR["Descuento"])

# Franjas

In [5]:
def convert_time(n):
    hour = n // 10000
    minute = (n % 10000) // 100
    second = n % 100  # Aquí corregimos el cálculo de los segundos
    period = "a. m." if hour < 12 else "p. m."

    if hour > 12:
        hour -= 12

    formatted_time = f"{hour:02d}:{minute:02d}:{second:02d} {period}"
    return formatted_time

In [6]:
BAYER_AR["Start Time"] = BAYER_AR["Hora"].apply(convert_time)

In [7]:
def categorize_franja(hora, Cat_Franj_AR):
    for index, row in Cat_Franj_AR.iterrows():
        franja = row['Franja']
        start_time = row['Inicio Franja']
        end_time = row['Fin Franja']
        if start_time <= hora <= end_time:
            return franja
    return "No Franja"

In [8]:
BAYER_AR['Franja'] = BAYER_AR["Hora"].apply(lambda x: categorize_franja(x, Cat_Franj_AR))


In [9]:
def convertir_fecha(fecha):
    str_fecha = str(fecha)
    dia = str_fecha[-2:]
    mes = str_fecha[-4:-2]
    anio = str_fecha[:-4]
    fecha_convertida = f"{dia}/{mes}/{anio}"
    return fecha_convertida

In [10]:
BAYER_AR["Fecha Convertida"] = BAYER_AR["Fecha"].apply(convertir_fecha)

In [11]:
BAYER_AR["Duración S"]=BAYER_AR["Volumen"]/BAYER_AR["Avisos"]

In [12]:
from datetime import datetime

# Función para convertir la cadena de fecha y obtener el número de la semana
def obtener_numero_semana(fecha_str):
    # Convierte la cadena de texto en un objeto date
    fecha = datetime.strptime(fecha_str, '%d/%m/%Y').date()
    
    # Obtiene el número de la semana usando isocalendar()
    numero_semana = fecha.isocalendar()[1]
    
    return numero_semana

In [13]:
BAYER_AR["Semana Num"]=BAYER_AR["Fecha Convertida"].apply(obtener_numero_semana)



In [14]:
BAYER_AR["Género"]=None
BAYER_AR["Televisora"]=None

In [15]:
BAYER_AR=BAYER_AR.merge(Cat_Targ_by_BC_AR,how="left",left_on=["BAYER CATEGORY","PRODUCT LINE"],right_on=["Bayer Category","Product Line"])
BAYER_AR=BAYER_AR.merge(Info_Targets_AR,how="left",left_on="Target_AR",right_on="Target_AR")

In [16]:
def obtener_trp(row):
    if pd.isna(row['Target_AR']):
        return np.nan
    else:
        return row[row['Target_AR']]

In [17]:
BAYER_AR['TRPs_AR'] = BAYER_AR.apply(obtener_trp, axis=1)

In [18]:
BAYER_AR["TRPs_AR 20s"]=BAYER_AR["TRPs_AR"]*(BAYER_AR["Duración S"]/20)

In [19]:
#BAYER_AR.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102839 entries, 0 to 102838
Data columns (total 64 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Sector               102839 non-null  object 
 1   Subsector            102839 non-null  object 
 2   Segmento             102839 non-null  object 
 3   Anunciante           102839 non-null  object 
 4   Marca                102839 non-null  object 
 5   Producto             102839 non-null  object 
 6   Tema                 102831 non-null  object 
 7   Material             102839 non-null  object 
 8   Tipo De Compra       102831 non-null  object 
 9   Tipo De Falla        102839 non-null  object 
 10  Localidad            102839 non-null  object 
 11  Clase De Vehiculo    102839 non-null  object 
 12  Vehiculo             102839 non-null  object 
 13  Programa             102364 non-null  object 
 14  Suplemento           90 non-null      object 
 15  Seccion          

In [20]:
Base_raw_AR.shape[0]==BAYER_AR.shape[0]

True

In [21]:
BAYER_AR.to_excel("BAYER_AR.xlsx")

# CHILE

# IMPORTACIÓN DE CATÁLOGOS.

Para conocer el medio, las televisoras, deflactación en inversión, TRPs 20', la categoría y subcategoría de nuestras marcas y productos propios y de la competencia.

In [23]:
ruta_catalogos_CHI = 'C:\\Users\\brandon.olvera\\OneDrive - insidemedia.net\\Documents\\Bayer\\2023\\LATAM\\Catalogos\\AG\\Catalogos CHI Layout_Dashboard.xlsx'
Catalogos_CHI = pd.read_excel(ruta_catalogos_CHI, sheet_name=None)
Cat_Franj_AR=Catalogos_AR['Cat_Franj_AR']
CatBrandPr_AR = Catalogos_AR['CatBrandPr_AR']
CatDescu_AR = Catalogos_AR['CatDescu_AR']
CatMediaTv_AR=Catalogos_AR['CatMediaTv_AR']
CatMediaNoTV_AR=Catalogos_AR['CatMediaNoTV_AR']
Cat_Targ_by_BC_AR=Catalogos_AR['Cat_Targ_by_BC_AR']
Info_Targets_AR=Catalogos_AR['Info_Targets_AR']
Cat_Anunc_AR=Catalogos_AR['Cat_Anunc_AR']

# COMPROBACIONES

In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)  # Ajusta la cantidad de decimales que deseas mostrar

pivot_table = BAYER_AR.pivot_table(index=["BAYER CATEGORY", "PRODUCT LINE","MARCA BAYER", "Marca", "Producto"], aggfunc='size')
pivot_table_reset = pivot_table.reset_index()
display(pivot_table_reset.style.hide(axis="index"))


BAYER CATEGORY,PRODUCT LINE,MARCA BAYER,Marca,Producto,0
DERMA,ANTIFUNGALS GYNAECOLOGICAL,1.000000,EMPECID,EMPECID/GYNO ANTIMICOTICO,39
DERMA,ANTIFUNGALS SKINS & NAILS,1.000000,EMPECID,EMPECID/ANTIMICOTICO,53
DERMA,ANTIFUNGALS SKINS & NAILS,1.000000,EMPECID,EMPECID/PIE ANTIMICOTICO,2214
NUTRITIONALS,MULTIVITAMINS ADULT,1.000000,SUPRADYN,SUPRADYN/ACTIVE VITAMINAS,62
NUTRITIONALS,VITAMIN C,1.000000,REDOXON,REDOXON/TRIPLE ACCION VITAMINA C,12
NUTRITIONALS,VITAMIN C,1.000000,REDOXON,REDOXON/VITAMINA C,2
PAIN,PAIN RELIEF ADULT,1.000000,ACTRON,ACTRON/RAPIDA ACCION ANALGESICO,4227
PAIN,PAIN RELIEF ADULT,1.000000,BAYER,BAYER/PARACETAMOL,762
PAIN,PAIN RELIEF TOPICAL,1.000000,ACTRON,ACTRON/GEL,3


In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)  # Ajusta la cantidad de decimales que deseas mostrar

pivot_table2 = BAYER_AR.pivot_table(index=["Año", "Clase De Vehiculo", "Descuento"], aggfunc='size')
pivot_table_reset2 = pivot_table2.reset_index()
display(pivot_table_reset2.style.hide(axis="index"))


Año,Clase De Vehiculo,Descuento,0
2023,Diarios Capital,0.220000,81
2023,Internet,0.000000,377
2023,Radio AM,0.650000,1452
2023,Radio FM,0.650000,2334
2023,Revistas,0.600000,9
2023,TV Cable,0.550000,23936
2023,TV Capital,0.800000,27356
2023,TV Interior,0.410000,47286
2023,Vía Pública,0.700000,8


In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)  # Ajusta la cantidad de decimales que deseas mostrar

pivot_table3 = BAYER_AR.pivot_table(index=["Clase De Vehiculo","Tipo de Medio"], aggfunc='size')
pivot_table_reset3 = pivot_table3.reset_index()
display(pivot_table_reset3.style.hide(axis="index"))


Clase De Vehiculo,Tipo de Medio,0
Diarios Capital,Print,81
Internet,Digital,377
Radio AM,Radio,1452
Radio FM,Radio,2334
Revistas,Print,9
TV Cable,PTV,23936
TV Capital,OTV,27356
TV Interior,OTV,47286
Vía Pública,Vía Pública,8


In [40]:
BAYER_AR[["Inversion","Miles","Descuento","Clase De Vehiculo","Miles con Descuento"]].head()

,Inversion,Miles,Descuento,Clase De Vehiculo,Miles con Descuento
0,2.76e+05,276.0,0.8,TV Capital,55.2
1,3.60e+05,360.0,0.8,TV Capital,72.0
2,1.04e+06,1035.0,0.8,TV Capital,207.0
3,1.12e+06,1125.0,0.8,TV Capital,225.0
4,1.35e+06,1350.0,0.8,TV Capital,270.0


In [41]:
BAYER_AR[BAYER_AR['Descuento'].isnull()][['Clase De Vehiculo','Tipo de Medio', 'Descuento']]



,Clase De Vehiculo,Tipo de Medio,Descuento


In [42]:
BAYER_AR.loc[BAYER_AR["Hora"] > 260000, ["Start Time", "Hora","Franja"]].head()




,Start Time,Hora,Franja
466,17:54:21 p. m.,295421,Late Night
467,17:54:48 p. m.,295448,Late Night
468,17:55:46 p. m.,295546,Late Night
469,17:56:19 p. m.,295619,Late Night
470,17:57:15 p. m.,295715,Late Night


In [43]:
BAYER_AR[["Fecha","Fecha Convertida"]].head()

,Fecha,Fecha Convertida
0,20230102,02/01/2023
1,20230102,02/01/2023
2,20230102,02/01/2023
3,20230102,02/01/2023
4,20230102,02/01/2023


In [51]:
BAYER_AR[["Volumen","Duración S"]].head()

,Volumen,Duración S
0,23.0,23.0
1,30.0,30.0
2,23.0,23.0
3,25.0,25.0
4,30.0,30.0


In [45]:
BAYER_AR[["A Amb 50 99","A Bc1c2 20 49 Nse06","A Bcd 20 Mas","A Bcd 20 49","F Bcd 20 49","Target_AR","TRPs_AR","TRPs_AR 20s"]].head(20)

,A Amb 50 99,A Bc1c2 20 49 Nse06,A Bcd 20 Mas,A Bcd 20 49,F Bcd 20 49,Target_AR,TRPs_AR,TRPs_AR 20s
0,0.12,0.00,0.13,0.14,0.27,A Bcd 20 Mas,0.13,0.15
1,0.20,0.00,0.16,0.14,0.27,NaN,NaN,NaN
2,1.87,0.58,1.48,1.26,1.26,A Bcd 20 Mas,1.48,1.70
3,1.87,0.58,1.41,1.15,1.26,A Bcd 20 Mas,1.41,1.76
4,1.87,0.58,1.41,1.15,1.26,NaN,NaN,NaN
5,1.87,0.58,1.41,1.15,1.26,A Bcd 20 49,1.15,1.21
6,1.87,0.29,1.53,1.33,1.45,F Bcd 20 49,1.45,1.81
7,1.87,0.29,1.53,1.33,1.45,A Bcd 20 Mas,1.53,2.07
8,1.87,0.29,1.53,1.33,1.45,A Bcd 20 Mas,1.53,1.68
9,1.87,0.29,1.53,1.33,1.45,A Bcd 20 Mas,1.53,1.38


In [ ]:
BAYER_AR[["A Amb 50 99","A Bc1c2 20 49 Nse06","A Bcd 20 Mas","A Bcd 20 49","F Bcd 20 49","Target_AR","TRPs_AR","TRPs_AR 20s"]].head(20)